In [1]:
import numpy as np# array oriented computing package
import import_ipynb
from itertools import repeat
from monoalphabetic_substitution import MonoAlphabeticCipher
from monoalphabetic_substitution import getRandomKey
import time 
from random import randint

importing Jupyter notebook from monoalphabetic_substitution.ipynb


## Histogram下的频率分析

首先咱们尝试最简单的方案,即简单的比较密文(Cipher Text)当中字母的频率与大数据结果的差异,并通过类似于$\chi^2$test的方法来评判

通过简单的频率分析后的密钥解密出来的结果与正常英文材料之间的差异.并通过调整密钥中字母以及符号的排列顺序使得结果更好.

### 0:准备

**0.1** 这一步首先LETTERS里面确认key包含的符号,也就是哪些符号需要参与替换

In [2]:
LETTERS= ' abcdefghijklmnopqrstuvwxyz'

**0.2** 这一步需要建立letterCount的值为0的字典用于之后计算频数,频率以及排序

In [3]:
letterCount = {' ':0,'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0, 'i': 0, 'j': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'o': 0, 'p': 0, 'q': 0, 'r': 0, 's': 0, 't': 0, 'u': 0, 'v': 0, 'w': 0, 'x': 0, 'y': 0, 'z': 0}

### 1:建立计算频数并排序的函数HistogramFreqExtract

In [4]:
def HistogramFreqExtract(message):
# 0.每次提取都要让leterCount归零
    letterCount = {' ':0,'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0, 'i': 0, 'j': 0, 'k': 0, 'l': 0, 'm': 0, 'n': 0, 'o': 0, 'p': 0, 'q': 0, 'r': 0, 's': 0, 't': 0, 'u': 0, 'v': 0, 'w': 0, 'x': 0, 'y': 0, 'z': 0}
# 1.算出文章中字母和空格的个数
    message=message.lower()
    message=message.replace('\n','')
    AlphaNumber=0
    for letter in message:
        if letter in LETTERS:
            AlphaNumber+=1
# 2.算出字母的频数
    for letter in message.lower():   
        if letter in LETTERS: 
            letterCount[letter] += 1 
# 3.算出字母的频率

    MessageCount=list(letterCount.values())
    MessageFreqArray=(np.asarray(MessageCount)/AlphaNumber)*100
    MessageFreqList=MessageFreqArray.tolist()# numpy中的函数,能把array变成list
    DictLetterFreq=dict(zip(list(letterCount.keys()),MessageFreqList)) #两个list通过zip()变成有序对,然后dict()变成字典 

    return DictLetterFreq

### 2:提取总体的频率作为保准频率

这一步需要一个英语文字(或者其他类似语言)的范本,数据量要足够的大,下面会把这个范本里面的字符的频率提取出来作为标准频率

这里采用的是一些经典的著名英文小说作为范本,你可以从这里免费下载text格式文件:www.gutenberg.org

In [5]:
with open('english.txt', 'r') as EnglishData:
    EnglishData=EnglishData.read()
HistoramNumFreq=HistogramFreqExtract(EnglishData)

### 3:建立评价函数

In [6]:
def HistogramMatchScore(message):
    LetterFreqList=list(HistogramFreqExtract(message).values())
    HistogramFreqList=list(HistoramNumFreq.values())
    LetterFreqArray=np.asarray(LetterFreqList)
    HistogramFreqArray=np.asarray(HistogramFreqList)
    Score=sum(np.absolute(LetterFreqArray-HistogramFreqArray))
    return Score

### 4:建立分析函数

**4.1** 一些前期准备

In [7]:
# 这个函数会用于排序
def getItemAtIndexOne(x):
    return x[1]
def getItemAtIndexZero(x):
    return x[0]

之所以要排序,是因为后面建立评价函数的时候,是样本的各个符号对应的频率和总体的大数据对应符号的频率做差.所以一定是**相同符号频率的差**,所以为了统一处理需要做一个排序

In [8]:
# 建立交换key中任何两个字符位置的函数
def Swap(key,symbolA,symbolB):
    ListKey=list(key)
    ListKey[key.find(symbolA)],ListKey[key.find(symbolB)]=ListKey[key.find(symbolB)],ListKey[key.find(symbolA)]
    key=''.join(ListKey)
    return key

下面这个函数是给密钥中的字符按照频率顺序排序,这将作为最后分析函数密钥的**初始值**

注意这个函数也是要用于bigram皮女郎分析函数当中的初始值的

In [9]:
# 这个函数是把字母按照频率由高到低进行排序的函数
# 定义这个函数有五个步骤
def FreqOrderkey(message):
    message=message.lower()
    message=message.replace('\n','')
# 1.得到按照频率由高到低的排序
    letterToFreq =HistogramFreqExtract(message)
    freqPairs = list(letterToFreq.items())
    freqPairs.sort(key=getItemAtIndexOne, reverse=True)
    
    freqOrder = []
    for freqPair in freqPairs:
        freqOrder.append(freqPair[0])
        

    return ''.join(freqOrder)

In [10]:
# 建立一个密钥转换函数:把按照频率对应的密钥,即对应' etaoi...'的密钥转换成对应' abcde....'的密钥
def convertKey(freqOrder):
    keyList=freqOrder
    etaoiList=list(' etaoinshrdlcumwfgypbvkjxqz')
    keyPairs=list(zip(keyList,etaoiList))
    keyPairs.sort(key=getItemAtIndexOne)
    
    key=[]
    for keyPair in keyPairs:      
        key.append(keyPair[0])
    return ''.join(key)

没有比要做histogram的迭代频率分析函数,因为直接简单的分析得到的结果通过评价函数得到的分数比明文还低,所以再做循环完全是没有任何必要的

## Bigram下的频率分析

### 第一步:从文件中导出数据,作为标准频率

**1.1**:从bigrams_from_Google_Web_Trillion_Word_Corpus.txt中调出bigrams全部的676种组合(不包含空格)

In [11]:
bigramDataList=list(open('bigrams_from_Google_Web_Trillion_Word_Corpus.txt','r'))
bigramLetters=[]
for i in range(len(bigramDataList)):
    bigramLetters.append(bigramDataList[i][:2])
#bigrams_from_Google_Web_Trillion_Word_Corpus.txt这个文件里面没有包含空格的组合
#,所以这里要把这部分添加进去,使得结果是27*27=729种组合
addBigram1=['  ',' a',' b',' c',' d',' e',' f',' g',' h',' i',' j',' k',' l',' m',' n',' o',' p',' q',' r',' s',' t',' u',' v',' w',' x',' y',' z']
addBigram2=['a ','b ','c ','d ','e ','f ','g ','h ','i ','j ','k ','l ','m ','n ','o ','p ','q ','r ','s ','t ','u ','v ','w ','x ','y ','z ']
bigramLetters=bigramLetters+addBigram1+addBigram2

### 第二步:建立能从材料得到bigram数据的函数

建立bigramCount函数,从材料中提取出bigram的频数,这一步暂且不计算频率和顺序

In [12]:
# 这个函数需要之前的那值为0的字典:bigramCountDict
# 本函数不需要排序是因为提取大数据和样本都是用的同样的bigramCountDict种的顺序
def bigramFreqExtract(message):
# 0.先让bigramCount归零
    bigramCount=dict(zip(bigramLetters,list(repeat(0,729))))
# 1.1整理材料,去除非LETTERS中的字符
    message=message.lower()
    message=message.replace('\n','')
# 1.2然后去掉材料中非字母部分(不包括空格,空格在字母表中),得到NewMessage
    NewMessage=[]
    for letter in message:
        if letter in LETTERS:  #区别是字母还是符号主要看是不是在字母表LETTERS当中
            NewMessage.append(letter)
# 2.得到bigrams
    bigramFromNewMessage=[]
    for j in range(len(NewMessage)-1): #因为后面是位移一位前进,所以这里要减一,这样倒数二位加一正好len的长度
        bigramFromNewMessage.append(''.join(NewMessage[j:j+2]))
# 3.得到频数
    for bigram in bigramFromNewMessage:
        bigramCount[bigram]+=1
# 4.得到频率
# 4.1先计算总数
    AlphaNumber=0
    for letter in message:
        if letter in LETTERS:
            AlphaNumber+=1
# 4.2计算频率
    bigramCountValues=list(bigramCount.values())
    bigramCountKeys=list(bigramCount.keys())
    bigramCountValuesArray=np.asarray(bigramCountValues)
    #和上面的bigramsNumFreq一致采用百分数
    bigramFreqArray=(bigramCountValuesArray/AlphaNumber)*100 
    bigramFreqList=bigramFreqArray.tolist()
    #这里得到字典是为了之后按照 bigramsNumFreq对应的顺序排序
    bigramExtractDict=dict(zip(bigramCountKeys,bigramFreqList)) 
# 5.提取数据
    bigramExtractDictPairs=list(bigramExtractDict.items())
    bigramExtractNumFreq=[]
    for bigramExtractDictPair in bigramExtractDictPairs:
        bigramExtractNumFreq.append(bigramExtractDictPair[1])
# 6.为了后面的使用
    
    return bigramExtractNumFreq


**2.1:** 用bgramFreqExtract函数导出EnglishData当中的bigram数据
EnglishData就是histogram分析里面的'MobyDick.txt','UncleTom’sCabin.txt'文字材料

In [13]:
# 先把bigram数据做好,保留下来,免得测量函数部分继续计算
bigramNumFreq=bigramFreqExtract(EnglishData)

### 第三步:建立测量函数

建立函数bigramMatchScore函数,用于对比给定材料和通常英文材料之间的差异,分数越大差异越大

In [14]:
def bigramMatchScore(message):
    bigramExtractArray=np.asarray(bigramFreqExtract(message)) # 从需要频率分析的材料中提取出来的bigram数据
    bigramDataArray=np.asarray(bigramNumFreq)                  # 从大数据文件中提取出来的bigram数据
    bigramScore=np.sum(np.absolute(bigramExtractArray-bigramDataArray))
    return bigramScore


### 第四步:建立bigramAnalysis函数

bigramAnalysis将由一个简单频率分析(Histogram)分析得到的key作为初始的key,然交换密钥中字母的顺序以得到更小的bigramScore

In [21]:
def bigramFreqAnalysis(Cipher,n):
    Cipher=Cipher.lower()
    Cipher=Cipher.replace('\n','')
    FreqKey=FreqOrderkey(Cipher) #这里的key是按照频率排序来的,并不是标准格式的key,所以需要转换
    key=convertKey(FreqKey)      #转换为标准格式的key
    Translation=MonoAlphabeticCipher(key,Cipher,'decrypt')
    Score=bigramMatchScore(Translation)
# 1.按照频率对Freqkey进行交换,优化破译结果
    for b in range(1,27):
        a=0
        while a+b<=26:
            FreqKey0=Swap(FreqKey,FreqKey[a],FreqKey[a+b]) #交换是按照频率顺序交换的,所以是交换Freqkey而不是标准key
            key0=convertKey(FreqKey0)
            Translation0=MonoAlphabeticCipher(key0,Cipher,'decrypt')
            Score0=bigramMatchScore(Translation0)
            a+=1
            if Score0<Score:
                FreqKey=FreqKey0
                Translation=Translation0
                Score=Score0
# 2.往往第一阶段完成以后结果还不是最好,所以接下来随机的进行替换,使得结果更好

    key=convertKey(FreqKey) #得到上一个阶段最后的密钥的标准格式
    
    for i in range(0,n):  # n是循环的次数
        RandChoiceA=randint(0,26) #得到1到27的随机整数
        RandChoiceB=randint(0,26)
        if RandChoiceA!=RandChoiceB:
            key0=Swap(key,key[RandChoiceA],key[RandChoiceB]) #随机交换密钥中两个字母的顺序
            Translation0=MonoAlphabeticCipher(key0,Cipher,'decrypt') # 解密
            Score0=bigramMatchScore(Translation0) # 评价
            if Score0<Score: #如果评价更好,得到新的密钥
                key=key0
                Translation=Translation0
                Score=Score0
                
    return Translation

## 测试

### 1:明文

明文不需要预处理,因为处理的函数写进了每一个Exract函数

In [17]:
PlainText='''
In cryptography, a substitution cipher is a method of encrypting by which units of plaintext are replaced with ciphertext, according to a fixed system; the "units" may be single letters (the most common), pairs of letters, triplets of letters, mixtures of the above, and so forth. The receiver deciphers the text by performing the inverse substitution.

Substitution ciphers can be compared with transposition ciphers. In a transposition cipher, the units of the plaintext are rearranged in a different and usually quite complex order, but the units themselves are left unchanged. By contrast, in a substitution cipher, the units of the plaintext are retained in the same sequence in the ciphertext, but the units themselves are altered.

There are a number of different types of substitution cipher. If the cipher operates on single letters, it is termed a simple substitution cipher; a cipher that operates on larger groups of letters is termed polygraphic. A monoalphabetic cipher uses fixed substitution over the entire message, whereas a polyalphabetic cipher uses a number of substitutions at different positions in the message, where a unit from the plaintext is mapped to one of several possibilities in the ciphertext and vice versa. 
'''
PlainText

'\nIn cryptography, a substitution cipher is a method of encrypting by which units of plaintext are replaced with ciphertext, according to a fixed system; the "units" may be single letters (the most common), pairs of letters, triplets of letters, mixtures of the above, and so forth. The receiver deciphers the text by performing the inverse substitution.\n\nSubstitution ciphers can be compared with transposition ciphers. In a transposition cipher, the units of the plaintext are rearranged in a different and usually quite complex order, but the units themselves are left unchanged. By contrast, in a substitution cipher, the units of the plaintext are retained in the same sequence in the ciphertext, but the units themselves are altered.\n\nThere are a number of different types of substitution cipher. If the cipher operates on single letters, it is termed a simple substitution cipher; a cipher that operates on larger groups of letters is termed polygraphic. A monoalphabetic cipher uses fixe

### 2:生成随机密钥并加密

In [18]:
key='rpflczmhbjetqd osiaynvkuwgx'
Cipher=MonoAlphabeticCipher(key,PlainText,'encrypt')
print('key:%s\n'%(key))
print('Ciphertext:\n%s'%(Cipher))

key:rpflczmhbjetqd osiaynvkuwgx

Ciphertext:
j rlagsnohapsbg,rpryvfynjnvnjo rljsbzarjyrprdznbocromrz lagsnj hrfgrubjlbrv jnyromrsqpj nzwnrpazrazsqplzcrujnbrljsbzanzwn,rplloacj hrnorprmjwzcrygynzd;rnbzr"v jny"rdpgrfzryj hqzrqznnzayr(nbzrdoynrloddo ),rspjayromrqznnzay,rnajsqznyromrqznnzay,rdjwnvazyromrnbzrpfokz,rp cryormoanb.rnbzrazlzjkzarczljsbzayrnbzrnzwnrfgrszamoadj hrnbzrj kzayzryvfynjnvnjo .yvfynjnvnjo rljsbzayrlp rfzrlodspazcrujnbrnap ysoyjnjo rljsbzay.rj rprnap ysoyjnjo rljsbza,rnbzrv jnyromrnbzrsqpj nzwnrpazrazpaap hzcrj rprcjmmzaz nrp crvyvpqqgrivjnzrlodsqzwroacza,rfvnrnbzrv jnyrnbzdyzqkzyrpazrqzmnrv lbp hzc.rfgrlo napyn,rj rpryvfynjnvnjo rljsbza,rnbzrv jnyromrnbzrsqpj nzwnrpazraznpj zcrj rnbzrypdzryzivz lzrj rnbzrljsbzanzwn,rfvnrnbzrv jnyrnbzdyzqkzyrpazrpqnzazc.nbzazrpazrpr vdfzaromrcjmmzaz nrngszyromryvfynjnvnjo rljsbza.rjmrnbzrljsbzaroszapnzyro ryj hqzrqznnzay,rjnrjyrnzadzcrpryjdsqzryvfynjnvnjo rljsbza;rprljsbzarnbpnroszapnzyro rqpahzarhaovsyromrqznnzayrjyrnzadzcrsoqghapsbjl.

### 3:测试bigram频率分析

In [19]:
start=time.time()
bigramDecrypt=bigramFreqAnalysis(Cipher,2000)
end=time.time()

convertKey(FreqOrderkey(Cipher))
simpleDecrypt=MonoAlphabeticCipher(convertKey(FreqOrderkey(Cipher)),Cipher,'decrypt')

print('破译结果:')                   
print(bigramDecrypt)
print('破译评价:')
print('==============破译时间:%s seconds'%(end-start))
print('==============明文得分:%s ponints'%(bigramMatchScore(PlainText)))
print('==============密文得分:%s ponints'%(bigramMatchScore(Cipher)))
print('==============histogram译文得分:%s ponints'%(bigramMatchScore(simpleDecrypt)))
print('==============bigram译文得分:%s ponints'%(bigramMatchScore(bigramDecrypt)))

破译结果:
in cryptography, a substitution cipher is a method of encrypting by which units of plaintekt are replaced with ciphertekt, according to a fiked system; the "units" may be single letters (the most common), pairs of letters, triplets of letters, miktures of the above, and so forth. the receiver deciphers the tekt by performing the inverse substitution.substitution ciphers can be compared with transposition ciphers. in a transposition cipher, the units of the plaintekt are rearranged in a different and usually quite complek order, but the units themselves are left unchanged. by contrast, in a substitution cipher, the units of the plaintekt are retained in the same sequence in the ciphertekt, but the units themselves are altered.there are a number of different types of substitution cipher. if the cipher operates on single letters, it is termed a simple substitution cipher; a cipher that operates on larger groups of letters is termed polygraphic. a monoalphabetic cipher uses fiked sub

In [ ]:
Translation=MonoAlphabeticCipher(key,Cipher,'decrypt')
Translation

In [ ]:
Score=bigramMatchScore(Translation)
Score

In [ ]:
FreqKey0=Swap(FreqKey,FreqKey[3],FreqKey[4])
FreqKey0

In [ ]:
key0=convertKey(FreqKey0)
key0

In [ ]:
Translation0=MonoAlphabeticCipher(key0,Cipher,'decrypt')
Translation0

In [ ]:
Score0=bigramMatchScore(Translation0)
Score0